In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, median_absolute_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
tf.random.set_seed(42)
np.random.seed(42)
file_path = "train_FD004_processed.csv"
df = pd.read_csv(file_path)

In [5]:
if df.isna().any().any():
    df = df.dropna()
max_rul = 130
df['RUL'] = df['RUL'].clip(upper=max_rul)

In [6]:
feature_cols = [col for col in df.columns if col.startswith('op_setting_') or col.startswith('sensor_measurement_')]
target_col = 'RUL'

In [7]:
window_size = 5
for col in feature_cols:
    df[f'{col}_rolling_mean'] = df.groupby('unit_number')[col].rolling(window=window_size, min_periods=1).mean().reset_index(level=0, drop=True)
    df[f'{col}_rolling_std'] = df.groupby('unit_number')[col].rolling(window=window_size, min_periods=1).std().reset_index(level=0, drop=True)
feature_cols += [col for col in df.columns if '_rolling_mean' in col or '_rolling_std' in col]

In [8]:
variances = df[feature_cols].var()
selected_features = variances[variances > 0.05].index.tolist()  # Stricter threshold (0.05 vs. 0.01)
df = df[['unit_number'] + selected_features + [target_col]]


In [9]:
scaler_features = StandardScaler()
scaler_target = StandardScaler()
df[selected_features] = scaler_features.fit_transform(df[selected_features])
df[target_col] = scaler_target.fit_transform(df[[target_col]])


In [10]:
WINDOW_SIZE = 30
def create_sequences(data, window_size, feature_cols, target_col):
    X, y = [], []
    for unit in data['unit_number'].unique():
        unit_data = data[data['unit_number'] == unit]
        feature_data = unit_data[feature_cols].values
        target_data = unit_data[target_col].values
        for i in range(len(unit_data) - window_size):
            X.append(feature_data[i:i+window_size])
            y.append(target_data[i+window_size])
    return np.array(X), np.array(y)

In [11]:
X, y = create_sequences(df, WINDOW_SIZE, selected_features, target_col)
mask = ~np.isnan(X).any(axis=(1, 2)) & ~np.isnan(y)
X, y = X[mask], y[mask]

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
input_shape = X.shape[1:]  
inputs = Input(shape=input_shape)
x = LSTM(64, return_sequences=True)(inputs) 
x = Dropout(0.3)(x)
x = LSTM(32)(x)  
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x) 
x = Dropout(0.3)(x)
outputs = Dense(1)(x)

In [14]:
model = Model(inputs, outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 30, 26)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 30, 64)         │        23,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,801 (143.75 KB)

 Trainable params: 36,801 (143.75 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)  # Reduced patience
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_lightweight_lstm_rul_model.keras', monitor='val_loss', save_best_only=True)


In [16]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,  # Reduced from 100
    batch_size=64,  # Increased from 32 for faster training
    callbacks=[early_stopping, lr_scheduler, checkpoint],
    verbose=1
)

Epoch 1/50
670/670 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - loss: 0.5311 - mae: 0.5901 - val_loss: 0.2674 - val_mae: 0.3990 - learning_rate: 0.0010
Epoch 2/50
670/670 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - loss: 0.3105 - mae: 0.4385 - val_loss: 0.2838 - val_mae: 0.4050 - learning_rate: 0.0010
Epoch 3/50
670/670 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - loss: 0.2809 - mae: 0.4130 - val_loss: 0.2408 - val_mae: 0.3636 - learning_rate: 0.0010
Epoch 4/50
670/670 ━━━━━━━━━━━━━━━━━━━━ 17s 25ms/step - loss: 0.2647 - mae: 0.3988 - val_loss: 0.2188 - val_mae: 0.3469 - learning_rate: 0.0010
Epoch 5/50
670/670 ━━━━━━━━━━━━━━━━━━━━ 16s 24ms/step - loss: 0.2547 - mae: 0.3903 - val_loss: 0.2282 - val_mae: 0.3475 - learning_rate: 0.0010
Epoch 6/50
670/670 ━━━━━━━━━━━━━━━━━━━━ 17s 25ms/step - loss: 0.2455 - mae: 0.3812 - val_loss: 0.2306 - val_mae: 0.3504 - learning_rate: 0.0010
Epoch 7/50
670/670 ━━━━━━━━━━━━━━━━━━━━ 17s 25ms/step - loss: 0.2350 - mae: 0.3720 - val_loss: 0.2311 - val_mae: 0.3588 - learning_rate:

In [17]:
y_pred = model.predict(X_val)
y_val_inv = scaler_target.inverse_transform(y_val.reshape(-1, 1)).flatten()
y_pred_inv = scaler_target.inverse_transform(y_pred).flatten()
mask = ~np.isnan(y_val_inv) & ~np.isnan(y_pred_inv)
y_val_inv, y_pred_inv = y_val_inv[mask], y_pred_inv[mask]

335/335 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [18]:
mse = mean_squared_error(y_val_inv, y_pred_inv)
mae = mean_absolute_error(y_val_inv, y_pred_inv)
rmse = np.sqrt(mse)
r2 = r2_score(y_val_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_val_inv, y_pred_inv)
medae = median_absolute_error(y_val_inv, y_pred_inv)

In [19]:
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"Median Absolute Error: {medae:.4f}")

MSE: 29.6718
MAE: 3.7931
RMSE: 5.4472
R² Score: 0.9842
MAPE: 2833655785592.68%
Median Absolute Error: 2.2541


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Lightweight LSTM Model Loss During Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('lightweight_lstm_training_loss_plot.png')
plt.close()

In [21]:
errors = y_val_inv - y_pred_inv
plt.figure(figsize=(10, 6))
sns.histplot(errors, bins=50, kde=True, color='blue')
plt.title('Distribution of RUL Prediction Errors (Lightweight LSTM)')
plt.xlabel('Prediction Error (Actual RUL - Predicted RUL)')
plt.ylabel('Frequency')
plt.grid(True)
plt.savefig('lightweight_prediction_error_distribution.png')
plt.close()

In [22]:
unit_ids = df['unit_number'].unique()
sample_unit = unit_ids[0]
unit_data = df[df['unit_number'] == sample_unit]
unit_X, unit_y = create_sequences(unit_data, WINDOW_SIZE, selected_features, target_col)
unit_pred = model.predict(unit_X)
unit_y_inv = scaler_target.inverse_transform(unit_y.reshape(-1, 1)).flatten()
unit_pred_inv = scaler_target.inverse_transform(unit_pred).flatten()


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


In [23]:
plt.figure(figsize=(12, 6))
plt.plot(unit_y_inv, label='Actual RUL', marker='o')
plt.plot(unit_pred_inv, label='Predicted RUL', marker='x')
plt.title(f'RUL Prediction for Unit {sample_unit} (Lightweight LSTM)')
plt.xlabel('Cycle')
plt.ylabel('RUL')
plt.legend()
plt.grid(True)
plt.savefig('lightweight_lstm_temporal_prediction_plot.png')
plt.close()

In [24]:
cumulative_error = np.cumsum(np.abs(errors))
plt.figure(figsize=(10, 6))
plt.plot(cumulative_error, color='red')
plt.title('Cumulative Absolute Prediction Error (Lightweight LSTM)')
plt.xlabel('Sample Index')
plt.ylabel('Cumulative Absolute Error')
plt.grid(True)
plt.savefig('lightweight_cumulative_error_plot.png')
plt.close()

In [25]:
model.save("lightweight_lstm_rul_model.keras")